In [1]:
import pandas as pd
from io import StringIO

def pv_data(data, capacity):
    """
    INPUTS: 
    * data (string): 1 day of 5-minute PV data copied from PVOutput.org (data only, no headers)
    * capacity (float): the PV installation capacity in kilowatts (kW)
    
    OUTPUT:
    * pandas.DataFrame object containing hourly normalized energy production data (kWh/kW)
    """

    # Data to be saved
    data_string = "Date	Time	Energy	Efficiency	Power	Average	Normalised	Temperature	Voltage	Energy Used	Power Used" + data

    # Remove units
    data_string = data_string.replace('kWh/kW','').replace('kWh','').replace('kW','').replace('W','').replace('V','')

    # Read data into a pandas DataFrame
    df = pd.read_csv(StringIO(data_string), sep='\t')

    # Combine Date and Time columns and convert to datetime
    df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%y %I:%M%p')

    # Drop unnecessary Date and Time columns
    df = df.drop(['Date', 'Time'], axis=1)

    # Set Datetime as the index
    df = df.set_index('Datetime')

    # Calculate last_time and first_time
    last_time = df.index[0]
    first_time = df.index[-1]
    cumulative_daily = df.loc[last_time, 'Energy']

    # Create a complete hourly index
    hourly_index = pd.date_range(start=df.index.min().replace(hour=0, minute=0, second=0), end=df.index.max().replace(hour=23, minute=0, second=0), freq='H')

    # Reindex the DataFrame to the complete hourly index and fill missing values with zeros
    df_resampled = df.reindex(hourly_index, fill_value=0)

    # Set Energy values before first_time to 0 and after last_time to the value at last_time
    df_resampled.loc[df_resampled.index < first_time, 'Energy'] = 0
    df_resampled.loc[df_resampled.index > last_time, 'Energy'] = cumulative_daily

    # Calculate hourly energy and normalize by capacity
    df_resampled['hourly_energy(kWh)'] = df_resampled['Energy'].shift(-1, fill_value=cumulative_daily) - df_resampled['Energy']
    df_resampled['normalized_hourly_energy_(kWh/kW)'] = df_resampled['hourly_energy(kWh)']/16.214

    # Display the resulting DataFrame
    return df_resampled[['normalized_hourly_energy_(kWh/kW)']]

In [2]:
capacity_kW = 16.94 # IN1
data = """
27/07/22	8:40PM	108.341kWh	6.396kWh/kW	0W	0W	0.000kW/kW	-	238.8V	-	-
27/07/22	8:35PM	108.341kWh	6.396kWh/kW	36W	36W	0.002kW/kW	-	238.5V	-	-
27/07/22	8:30PM	108.338kWh	6.395kWh/kW	72W	72W	0.004kW/kW	-	238.4V	-	-
27/07/22	8:25PM	108.332kWh	6.395kWh/kW	132W	132W	0.008kW/kW	-	238.6V	-	-
27/07/22	8:20PM	108.321kWh	6.394kWh/kW	168W	168W	0.010kW/kW	-	238.5V	-	-
27/07/22	8:15PM	108.307kWh	6.394kWh/kW	228W	228W	0.013kW/kW	-	238.6V	-	-
27/07/22	8:10PM	108.288kWh	6.392kWh/kW	288W	288W	0.017kW/kW	-	241.1V	-	-
27/07/22	8:05PM	108.264kWh	6.391kWh/kW	396W	396W	0.023kW/kW	-	240.7V	-	-
27/07/22	8:00PM	108.231kWh	6.389kWh/kW	444W	444W	0.026kW/kW	-	240.9V	-	-
27/07/22	7:55PM	108.194kWh	6.387kWh/kW	528W	528W	0.031kW/kW	-	240.5V	-	-
27/07/22	7:50PM	108.150kWh	6.384kWh/kW	708W	708W	0.042kW/kW	-	240.0V	-	-
27/07/22	7:45PM	108.091kWh	6.381kWh/kW	912W	912W	0.054kW/kW	-	239.2V	-	-
27/07/22	7:40PM	108.015kWh	6.376kWh/kW	1,152W	1,152W	0.068kW/kW	-	239.8V	-	-
27/07/22	7:35PM	107.919kWh	6.371kWh/kW	1,380W	1,380W	0.081kW/kW	-	239.8V	-	-
27/07/22	7:30PM	107.804kWh	6.364kWh/kW	1,596W	1,596W	0.094kW/kW	-	239.8V	-	-
27/07/22	7:25PM	107.671kWh	6.356kWh/kW	1,800W	1,800W	0.106kW/kW	-	240.0V	-	-
27/07/22	7:20PM	107.521kWh	6.347kWh/kW	2,052W	2,052W	0.121kW/kW	-	240.0V	-	-
27/07/22	7:15PM	107.350kWh	6.337kWh/kW	2,256W	2,256W	0.133kW/kW	-	239.8V	-	-
27/07/22	7:10PM	107.162kWh	6.326kWh/kW	2,532W	2,532W	0.149kW/kW	-	240.4V	-	-
27/07/22	7:05PM	106.951kWh	6.314kWh/kW	2,748W	2,748W	0.162kW/kW	-	240.5V	-	-
27/07/22	7:00PM	106.722kWh	6.300kWh/kW	2,988W	2,988W	0.176kW/kW	-	240.7V	-	-
27/07/22	6:55PM	106.473kWh	6.285kWh/kW	3,228W	3,228W	0.191kW/kW	-	240.1V	-	-
27/07/22	6:50PM	106.204kWh	6.269kWh/kW	3,528W	3,528W	0.208kW/kW	-	240.4V	-	-
27/07/22	6:45PM	105.910kWh	6.252kWh/kW	3,828W	3,828W	0.226kW/kW	-	240.2V	-	-
27/07/22	6:40PM	105.591kWh	6.233kWh/kW	4,080W	4,080W	0.241kW/kW	-	240.3V	-	-
27/07/22	6:35PM	105.251kWh	6.213kWh/kW	4,356W	4,356W	0.257kW/kW	-	240.1V	-	-
27/07/22	6:30PM	104.888kWh	6.192kWh/kW	4,656W	4,656W	0.275kW/kW	-	240.2V	-	-
27/07/22	6:25PM	104.500kWh	6.169kWh/kW	4,896W	4,896W	0.289kW/kW	-	240.1V	-	-
27/07/22	6:20PM	104.092kWh	6.145kWh/kW	5,172W	5,172W	0.305kW/kW	-	240.1V	-	-
27/07/22	6:15PM	103.661kWh	6.119kWh/kW	5,448W	5,448W	0.322kW/kW	-	240.4V	-	-
27/07/22	6:10PM	103.207kWh	6.093kWh/kW	5,736W	5,736W	0.339kW/kW	-	240.4V	-	-
27/07/22	6:05PM	102.729kWh	6.064kWh/kW	5,988W	5,988W	0.353kW/kW	-	240.3V	-	-
27/07/22	6:00PM	102.230kWh	6.035kWh/kW	6,216W	6,216W	0.367kW/kW	-	239.9V	-	-
27/07/22	5:55PM	101.712kWh	6.004kWh/kW	6,480W	6,480W	0.383kW/kW	-	239.3V	-	-
27/07/22	5:50PM	101.172kWh	5.972kWh/kW	6,780W	6,780W	0.400kW/kW	-	239.4V	-	-
27/07/22	5:45PM	100.607kWh	5.939kWh/kW	7,080W	7,080W	0.418kW/kW	-	239.1V	-	-
27/07/22	5:40PM	100.017kWh	5.904kWh/kW	7,272W	7,272W	0.429kW/kW	-	239.4V	-	-
27/07/22	5:35PM	99.411kWh	5.868kWh/kW	7,512W	7,512W	0.443kW/kW	-	239.3V	-	-
27/07/22	5:30PM	98.785kWh	5.831kWh/kW	7,728W	7,728W	0.456kW/kW	-	239.3V	-	-
27/07/22	5:25PM	98.141kWh	5.793kWh/kW	7,968W	7,968W	0.470kW/kW	-	239.2V	-	-
27/07/22	5:20PM	97.477kWh	5.754kWh/kW	8,208W	8,208W	0.485kW/kW	-	239.1V	-	-
27/07/22	5:15PM	96.793kWh	5.714kWh/kW	8,592W	8,592W	0.507kW/kW	-	239.3V	-	-
27/07/22	5:10PM	96.077kWh	5.672kWh/kW	8,724W	8,724W	0.515kW/kW	-	239.3V	-	-
27/07/22	5:05PM	95.350kWh	5.629kWh/kW	8,928W	8,928W	0.527kW/kW	-	238.9V	-	-
27/07/22	5:00PM	94.606kWh	5.585kWh/kW	9,120W	9,120W	0.538kW/kW	-	239.8V	-	-
27/07/22	4:55PM	93.846kWh	5.540kWh/kW	9,264W	9,264W	0.547kW/kW	-	239.2V	-	-
27/07/22	4:50PM	93.074kWh	5.494kWh/kW	9,504W	9,504W	0.561kW/kW	-	239.4V	-	-
27/07/22	4:45PM	92.282kWh	5.448kWh/kW	9,660W	9,660W	0.570kW/kW	-	239.3V	-	-
27/07/22	4:40PM	91.477kWh	5.400kWh/kW	9,900W	9,900W	0.584kW/kW	-	239.4V	-	-
27/07/22	4:35PM	90.652kWh	5.351kWh/kW	10,056W	10,056W	0.594kW/kW	-	241.4V	-	-
27/07/22	4:30PM	89.814kWh	5.302kWh/kW	10,212W	10,212W	0.603kW/kW	-	241.4V	-	-
27/07/22	4:25PM	88.963kWh	5.252kWh/kW	10,428W	10,428W	0.616kW/kW	-	240.9V	-	-
27/07/22	4:20PM	88.094kWh	5.200kWh/kW	10,668W	10,668W	0.630kW/kW	-	241.0V	-	-
27/07/22	4:15PM	87.205kWh	5.148kWh/kW	10,776W	10,776W	0.636kW/kW	-	241.4V	-	-
27/07/22	4:10PM	86.307kWh	5.095kWh/kW	10,944W	10,944W	0.646kW/kW	-	241.3V	-	-
27/07/22	4:05PM	85.395kWh	5.041kWh/kW	11,208W	11,208W	0.662kW/kW	-	241.2V	-	-
27/07/22	4:00PM	84.461kWh	4.986kWh/kW	11,292W	11,292W	0.667kW/kW	-	241.4V	-	-
27/07/22	3:55PM	83.520kWh	4.930kWh/kW	11,364W	11,364W	0.671kW/kW	-	241.1V	-	-
27/07/22	3:50PM	82.573kWh	4.874kWh/kW	11,484W	11,484W	0.678kW/kW	-	240.0V	-	-
27/07/22	3:45PM	81.616kWh	4.818kWh/kW	11,640W	11,640W	0.687kW/kW	-	240.4V	-	-
27/07/22	3:40PM	80.646kWh	4.761kWh/kW	11,724W	11,724W	0.692kW/kW	-	240.3V	-	-
27/07/22	3:35PM	79.669kWh	4.703kWh/kW	11,868W	11,868W	0.701kW/kW	-	240.4V	-	-
27/07/22	3:30PM	78.680kWh	4.645kWh/kW	12,000W	12,000W	0.708kW/kW	-	240.2V	-	-
27/07/22	3:25PM	77.680kWh	4.586kWh/kW	12,168W	12,168W	0.718kW/kW	-	240.2V	-	-
27/07/22	3:20PM	76.666kWh	4.526kWh/kW	12,264W	12,264W	0.724kW/kW	-	240.0V	-	-
27/07/22	3:15PM	75.644kWh	4.465kWh/kW	12,384W	12,384W	0.731kW/kW	-	240.2V	-	-
27/07/22	3:10PM	74.612kWh	4.404kWh/kW	12,480W	12,480W	0.737kW/kW	-	240.5V	-	-
27/07/22	3:05PM	73.572kWh	4.343kWh/kW	12,492W	12,492W	0.737kW/kW	-	239.9V	-	-
27/07/22	3:00PM	72.531kWh	4.282kWh/kW	12,552W	12,552W	0.741kW/kW	-	240.0V	-	-
27/07/22	2:55PM	71.485kWh	4.220kWh/kW	12,660W	12,660W	0.747kW/kW	-	239.8V	-	-
27/07/22	2:50PM	70.430kWh	4.158kWh/kW	12,696W	12,696W	0.749kW/kW	-	240.2V	-	-
27/07/22	2:45PM	69.372kWh	4.095kWh/kW	12,780W	12,780W	0.754kW/kW	-	239.5V	-	-
27/07/22	2:40PM	68.307kWh	4.032kWh/kW	12,792W	12,792W	0.755kW/kW	-	239.5V	-	-
27/07/22	2:35PM	67.241kWh	3.969kWh/kW	12,840W	12,840W	0.758kW/kW	-	239.3V	-	-
27/07/22	2:30PM	66.171kWh	3.906kWh/kW	12,924W	12,924W	0.763kW/kW	-	239.6V	-	-
27/07/22	2:25PM	65.094kWh	3.843kWh/kW	12,984W	12,984W	0.766kW/kW	-	239.2V	-	-
27/07/22	2:20PM	64.012kWh	3.779kWh/kW	13,056W	13,056W	0.771kW/kW	-	238.9V	-	-
27/07/22	2:15PM	62.924kWh	3.715kWh/kW	13,044W	13,044W	0.770kW/kW	-	239.0V	-	-
27/07/22	2:10PM	61.837kWh	3.650kWh/kW	13,008W	13,008W	0.768kW/kW	-	239.0V	-	-
27/07/22	2:05PM	60.753kWh	3.586kWh/kW	13,080W	13,080W	0.772kW/kW	-	239.0V	-	-
27/07/22	2:00PM	59.663kWh	3.522kWh/kW	13,200W	13,200W	0.779kW/kW	-	239.5V	-	-
27/07/22	1:55PM	58.563kWh	3.457kWh/kW	13,236W	13,236W	0.781kW/kW	-	238.8V	-	-
27/07/22	1:50PM	57.460kWh	3.392kWh/kW	13,188W	13,188W	0.779kW/kW	-	238.6V	-	-
27/07/22	1:45PM	56.361kWh	3.327kWh/kW	13,236W	13,236W	0.781kW/kW	-	238.7V	-	-
27/07/22	1:40PM	55.258kWh	3.262kWh/kW	13,332W	13,332W	0.787kW/kW	-	238.9V	-	-
27/07/22	1:35PM	54.147kWh	3.196kWh/kW	13,428W	13,428W	0.793kW/kW	-	239.0V	-	-
27/07/22	1:30PM	53.028kWh	3.130kWh/kW	13,416W	13,416W	0.792kW/kW	-	239.2V	-	-
27/07/22	1:25PM	51.910kWh	3.064kWh/kW	13,428W	13,428W	0.793kW/kW	-	239.0V	-	-
27/07/22	1:20PM	50.791kWh	2.998kWh/kW	13,416W	13,416W	0.792kW/kW	-	239.0V	-	-
27/07/22	1:15PM	49.673kWh	2.932kWh/kW	13,392W	13,392W	0.791kW/kW	-	239.0V	-	-
27/07/22	1:10PM	48.557kWh	2.866kWh/kW	13,296W	13,296W	0.785kW/kW	-	235.6V	-	-
27/07/22	1:05PM	47.449kWh	2.801kWh/kW	13,272W	13,272W	0.783kW/kW	-	235.8V	-	-
27/07/22	1:00PM	46.343kWh	2.736kWh/kW	13,152W	13,152W	0.776kW/kW	-	235.9V	-	-
27/07/22	12:55PM	45.247kWh	2.671kWh/kW	13,068W	13,068W	0.771kW/kW	-	235.8V	-	-
27/07/22	12:50PM	44.158kWh	2.607kWh/kW	13,104W	13,104W	0.774kW/kW	-	236.1V	-	-
27/07/22	12:45PM	43.066kWh	2.542kWh/kW	13,044W	13,044W	0.770kW/kW	-	236.0V	-	-
27/07/22	12:40PM	41.979kWh	2.478kWh/kW	13,008W	13,008W	0.768kW/kW	-	236.2V	-	-
27/07/22	12:35PM	40.895kWh	2.414kWh/kW	12,960W	12,960W	0.765kW/kW	-	236.6V	-	-
27/07/22	12:30PM	39.815kWh	2.350kWh/kW	12,852W	12,852W	0.759kW/kW	-	236.2V	-	-
27/07/22	12:25PM	38.744kWh	2.287kWh/kW	12,792W	12,792W	0.755kW/kW	-	236.8V	-	-
27/07/22	12:20PM	37.678kWh	2.224kWh/kW	12,708W	12,708W	0.750kW/kW	-	236.8V	-	-
27/07/22	12:15PM	36.619kWh	2.162kWh/kW	12,708W	12,708W	0.750kW/kW	-	236.8V	-	-
27/07/22	12:10PM	35.560kWh	2.099kWh/kW	12,612W	12,612W	0.745kW/kW	-	236.7V	-	-
27/07/22	12:05PM	34.509kWh	2.037kWh/kW	12,504W	12,504W	0.738kW/kW	-	236.8V	-	-
27/07/22	12:00PM	33.467kWh	1.976kWh/kW	12,348W	12,348W	0.729kW/kW	-	236.7V	-	-
27/07/22	11:55AM	32.438kWh	1.915kWh/kW	12,240W	12,240W	0.723kW/kW	-	236.5V	-	-
27/07/22	11:50AM	31.418kWh	1.855kWh/kW	12,180W	12,180W	0.719kW/kW	-	236.0V	-	-
27/07/22	11:45AM	30.403kWh	1.795kWh/kW	11,952W	11,952W	0.706kW/kW	-	236.7V	-	-
27/07/22	11:40AM	29.407kWh	1.736kWh/kW	11,856W	11,856W	0.700kW/kW	-	236.0V	-	-
27/07/22	11:35AM	28.419kWh	1.678kWh/kW	11,772W	11,772W	0.695kW/kW	-	236.2V	-	-
27/07/22	11:30AM	27.438kWh	1.620kWh/kW	11,628W	11,628W	0.686kW/kW	-	236.0V	-	-
27/07/22	11:25AM	26.469kWh	1.563kWh/kW	11,496W	11,496W	0.679kW/kW	-	236.5V	-	-
27/07/22	11:20AM	25.511kWh	1.506kWh/kW	11,292W	11,292W	0.667kW/kW	-	236.2V	-	-
27/07/22	11:15AM	24.570kWh	1.450kWh/kW	11,172W	11,172W	0.660kW/kW	-	236.6V	-	-
27/07/22	11:10AM	23.639kWh	1.395kWh/kW	11,028W	11,028W	0.651kW/kW	-	236.8V	-	-
27/07/22	11:05AM	22.720kWh	1.341kWh/kW	10,920W	10,920W	0.645kW/kW	-	236.7V	-	-
27/07/22	11:00AM	21.810kWh	1.287kWh/kW	10,740W	10,740W	0.634kW/kW	-	236.5V	-	-
27/07/22	10:55AM	20.915kWh	1.235kWh/kW	10,608W	10,608W	0.626kW/kW	-	236.6V	-	-
27/07/22	10:50AM	20.031kWh	1.182kWh/kW	10,404W	10,404W	0.614kW/kW	-	236.1V	-	-
27/07/22	10:45AM	19.164kWh	1.131kWh/kW	10,224W	10,224W	0.604kW/kW	-	236.4V	-	-
27/07/22	10:40AM	18.312kWh	1.081kWh/kW	10,020W	10,020W	0.591kW/kW	-	236.6V	-	-
27/07/22	10:35AM	17.477kWh	1.032kWh/kW	9,828W	9,828W	0.580kW/kW	-	236.6V	-	-
27/07/22	10:30AM	16.658kWh	0.983kWh/kW	9,648W	9,648W	0.570kW/kW	-	236.6V	-	-
27/07/22	10:25AM	15.854kWh	0.936kWh/kW	9,456W	9,456W	0.558kW/kW	-	236.4V	-	-
27/07/22	10:20AM	15.066kWh	0.889kWh/kW	9,252W	9,252W	0.546kW/kW	-	236.1V	-	-
27/07/22	10:15AM	14.295kWh	0.844kWh/kW	9,072W	9,072W	0.536kW/kW	-	234.6V	-	-
27/07/22	10:10AM	13.539kWh	0.799kWh/kW	8,856W	8,856W	0.523kW/kW	-	235.2V	-	-
27/07/22	10:05AM	12.801kWh	0.756kWh/kW	8,628W	8,628W	0.509kW/kW	-	235.0V	-	-
27/07/22	10:00AM	12.082kWh	0.713kWh/kW	8,400W	8,400W	0.496kW/kW	-	235.7V	-	-
27/07/22	9:55AM	11.382kWh	0.672kWh/kW	8,172W	8,172W	0.482kW/kW	-	235.1V	-	-
27/07/22	9:50AM	10.701kWh	0.632kWh/kW	7,836W	7,836W	0.463kW/kW	-	235.5V	-	-
27/07/22	9:45AM	10.048kWh	0.593kWh/kW	7,572W	7,572W	0.447kW/kW	-	234.5V	-	-
27/07/22	9:40AM	9.417kWh	0.556kWh/kW	7,332W	7,332W	0.433kW/kW	-	233.9V	-	-
27/07/22	9:35AM	8.806kWh	0.520kWh/kW	7,092W	7,092W	0.419kW/kW	-	234.4V	-	-
27/07/22	9:30AM	8.215kWh	0.485kWh/kW	6,792W	6,792W	0.401kW/kW	-	235.3V	-	-
27/07/22	9:25AM	7.649kWh	0.452kWh/kW	6,588W	6,588W	0.389kW/kW	-	234.4V	-	-
27/07/22	9:20AM	7.100kWh	0.419kWh/kW	6,336W	6,336W	0.374kW/kW	-	235.2V	-	-
27/07/22	9:15AM	6.572kWh	0.388kWh/kW	6,108W	6,108W	0.361kW/kW	-	235.3V	-	-
27/07/22	9:10AM	6.063kWh	0.358kWh/kW	5,892W	5,892W	0.348kW/kW	-	236.6V	-	-
27/07/22	9:05AM	5.572kWh	0.329kWh/kW	5,616W	5,616W	0.332kW/kW	-	236.8V	-	-
27/07/22	9:00AM	5.104kWh	0.301kWh/kW	5,364W	5,364W	0.317kW/kW	-	236.9V	-	-
27/07/22	8:55AM	4.657kWh	0.275kWh/kW	5,088W	5,088W	0.300kW/kW	-	236.8V	-	-
27/07/22	8:50AM	4.233kWh	0.250kWh/kW	4,836W	4,836W	0.285kW/kW	-	236.4V	-	-
27/07/22	8:45AM	3.830kWh	0.226kWh/kW	4,560W	4,560W	0.269kW/kW	-	235.0V	-	-
27/07/22	8:40AM	3.450kWh	0.204kWh/kW	4,272W	4,272W	0.252kW/kW	-	235.6V	-	-
27/07/22	8:35AM	3.094kWh	0.183kWh/kW	3,996W	3,996W	0.236kW/kW	-	235.7V	-	-
27/07/22	8:30AM	2.761kWh	0.163kWh/kW	3,720W	3,720W	0.220kW/kW	-	235.6V	-	-
27/07/22	8:25AM	2.451kWh	0.145kWh/kW	3,456W	3,456W	0.204kW/kW	-	235.4V	-	-
27/07/22	8:20AM	2.163kWh	0.128kWh/kW	3,192W	3,192W	0.188kW/kW	-	235.8V	-	-
27/07/22	8:15AM	1.897kWh	0.112kWh/kW	2,928W	2,928W	0.173kW/kW	-	234.4V	-	-
27/07/22	8:10AM	1.653kWh	0.098kWh/kW	2,664W	2,664W	0.157kW/kW	-	234.3V	-	-
27/07/22	8:05AM	1.431kWh	0.084kWh/kW	2,424W	2,424W	0.143kW/kW	-	234.1V	-	-
27/07/22	8:00AM	1.229kWh	0.073kWh/kW	2,172W	2,172W	0.128kW/kW	-	233.7V	-	-
27/07/22	7:55AM	1.048kWh	0.062kWh/kW	1,944W	1,944W	0.115kW/kW	-	234.9V	-	-
27/07/22	7:50AM	0.886kWh	0.052kWh/kW	1,704W	1,704W	0.101kW/kW	-	235.1V	-	-
27/07/22	7:45AM	0.744kWh	0.044kWh/kW	1,476W	1,476W	0.087kW/kW	-	235.2V	-	-
27/07/22	7:40AM	0.621kWh	0.037kWh/kW	1,284W	1,284W	0.076kW/kW	-	235.2V	-	-
27/07/22	7:35AM	0.514kWh	0.030kWh/kW	1,104W	1,104W	0.065kW/kW	-	235.5V	-	-
27/07/22	7:30AM	0.422kWh	0.025kWh/kW	936W	936W	0.055kW/kW	-	236.1V	-	-
27/07/22	7:25AM	0.344kWh	0.020kWh/kW	780W	780W	0.046kW/kW	-	236.9V	-	-
27/07/22	7:20AM	0.279kWh	0.016kWh/kW	660W	660W	0.039kW/kW	-	234.7V	-	-
27/07/22	7:15AM	0.224kWh	0.013kWh/kW	540W	540W	0.032kW/kW	-	234.8V	-	-
27/07/22	7:10AM	0.179kWh	0.011kWh/kW	468W	468W	0.028kW/kW	-	234.9V	-	-
27/07/22	7:05AM	0.140kWh	0.008kWh/kW	396W	396W	0.023kW/kW	-	235.4V	-	-
27/07/22	7:00AM	0.107kWh	0.006kWh/kW	336W	336W	0.020kW/kW	-	235.7V	-	-
27/07/22	6:55AM	0.079kWh	0.005kWh/kW	288W	288W	0.017kW/kW	-	236.3V	-	-
27/07/22	6:50AM	0.055kWh	0.003kWh/kW	240W	240W	0.014kW/kW	-	236.3V	-	-
27/07/22	6:45AM	0.035kWh	0.002kWh/kW	180W	180W	0.011kW/kW	-	236.3V	-	-
27/07/22	6:40AM	0.020kWh	0.001kWh/kW	132W	132W	0.008kW/kW	-	236.8V	-	-
27/07/22	6:35AM	0.009kWh	0.001kWh/kW	84W	84W	0.005kW/kW	-	236.8V	-	-
27/07/22	6:30AM	0.002kWh	0.000kWh/kW	24W	24W	0.001kW/kW	-	237.2V	-	-
27/07/22	6:25AM	0.000kWh	0.000kWh/kW	0W	-	-	-	237.4V	-	-"""
pv_data(data=data, capacity=capacity_kW)

,normalized_hourly_energy_(kWh/kW)
2022-07-27 00:00:00,0.000000
2022-07-27 01:00:00,0.000000
2022-07-27 02:00:00,0.000000
2022-07-27 03:00:00,0.000000
2022-07-27 04:00:00,0.000000
2022-07-27 05:00:00,0.000000
2022-07-27 06:00:00,0.006599
2022-07-27 07:00:00,0.069199
2022-07-27 08:00:00,0.238991
2022-07-27 09:00:00,0.430369
